In [4]:
import pandas as pd
import os
import numpy as np

from analysis import compile_metrics
from pprint import pprint

import glob

In [5]:
all_languages = ["Bulgarian", "Danish", "German", "English", "Spanish", "Persian", "Hungarian", "Italian", "Dutch", "Polish", "Portuguese", "Romanian", "Slovak", "Slovenian", "Swedish"]

In [10]:
# Need to look at the results of the hyperparameter search

import importlib
importlib.reload(compile_metrics)

# Need to compare for different versions

from collections import defaultdict
version_metrics = defaultdict(int)
best_version_keys = defaultdict(str)
# Total expected: 12
represented_metrics = defaultdict(set)

for seed in [42]:

    seed_path = f'../experiments/crosslingual_private_grid/seed={seed}'
    hyperparameter_search_path = os.path.join(seed_path, '0-shot-finetune-freeze-1', 'bert-base-multilingual-cased')

    current_version_metrics = {}
    hyperparameter_version_base_path = f'{hyperparameter_search_path}/*/version_0'
    
    hyperparameter_paths = glob.glob(hyperparameter_version_base_path)
    
    for path in hyperparameter_paths:
        represented_metrics[seed] |= {path.split('/')[-2]}
        best_validation_entry = compile_metrics.load_evaluation_results(path, 'val')
        best_validation_accuracy = best_validation_entry['val_English_acc']
        current_version_metrics[path] = best_validation_accuracy
    
    all_keys = sorted(current_version_metrics.keys())
    
    # Look for the maximum value amongst the grid search values
    sorted_key_indices = np.argsort(list(map(lambda entry : current_version_metrics[entry], all_keys)))
    best_key_index = sorted_key_indices[-1]
    best_key = all_keys[best_key_index]
    best_version_keys[seed] = best_key
    
    test_accuracies = compile_metrics.generate_accuracy_dict(best_key, 'tst', all_languages)
    paper_difference = compile_metrics.compare_results_to_paper(test_accuracies)
    
    version_metrics[seed] = (test_accuracies, paper_difference)
    

In [12]:
version_metrics

defaultdict(int,
            {42: ({'Bulgarian': 87.44120788574219,
               'Danish': 88.85111999511719,
               'German': 90.61707305908203,
               'English': 96.93975067138672,
               'Spanish': 87.2878189086914,
               'Persian': 72.51717376708984,
               'Hungarian': 82.93206024169922,
               'Italian': 91.47789764404297,
               'Dutch': 75.5525131225586,
               'Polish': 85.78205871582031,
               'Portuguese': 88.57642364501953,
               'Romanian': 83.96122741699219,
               'Slovak': 84.01200866699219,
               'Slovenian': 84.73298645019531,
               'Swedish': 91.76031494140625},
              {'Bulgarian': 0.041207885742181816,
               'Danish': 0.5511199951171903,
               'German': 0.8170730590820341,
               'English': -0.16024932861327557,
               'Spanish': 2.0878189086914034,
               'Persian': -0.2828262329101534,
               'Hung

## compile their val metrics for use in baseline comparisons

In [11]:
# Need to look at the results of the hyperparameter search

print(best_version_keys[42])
best_validation_accuracy = compile_metrics.load_evaluation_results(best_version_keys[42], 'val')

import pprint
pprint.pprint(best_validation_accuracy)

# Note that will need to evaluate baseline accuracy on the other languages' val set.

../experiments/crosslingual_private_grid/seed=42/0-shot-finetune-freeze-1/bert-base-multilingual-cased/bs32-lr5e-5-ep4/version_0
{'step': 1571,
 'val_English_acc': 97.12104034423828,
 'val_English_loss': 0.12849466502666473,
 'val_acc': 97.12104034423828,
 'val_loss': 0.12849466502666473}
